# 0301 축 기준
- apply : function = 각 행이나 열에 적용할 함수
- axis : 1=열, 0=행
- row : True : ndarray, False" Series
- result_type = {expand/reduce/broadcast}
- expand는 배열형태를 기준으로 열을 확장
- reduce는 그대로 series
- broadcast는 기존 열 형식대로 확장하여 반환함

In [2]:
import pandas as pd
import numpy as np

In [3]:
col= ['col1', 'col2', 'col3']
row = ['row1', 'row2', 'row3']
data = [[1,2,3], [4,5,6], [7,8,9]]
df = pd.DataFrame(data=data, index=row, columns=col)
print(df)

      col1  col2  col3
row1     1     2     3
row2     4     5     6
row3     7     8     9


In [4]:
#func의 성질에 따른 차이 func이 sqrt처럼 축에 대해 계산할 수 없는 형식이라면 각 요소에 적용됨
print(df.apply(np.sqrt))

          col1      col2      col3
row1  1.000000  1.414214  1.732051
row2  2.000000  2.236068  2.449490
row3  2.645751  2.828427  3.000000


In [5]:
print(df.apply(np.sum)) #축 기준으로 연산 수행

col1    12
col2    15
col3    18
dtype: int64


In [6]:
# axis에 따른 차이
print(df.apply(np.prod, axis=0)) #행에 대해 연산하는데 1열을 다 곱함.

col1     28
col2     80
col3    162
dtype: int64


In [8]:
# axis에 따른 차이 axis=1일 때 열에 대해 연산
print(df.apply(np.prod, axis=1))

row1      6
row2    120
row3    504
dtype: int64


In [10]:
#result_type에 따른 차이
#lambda를 사용하여 기존 datagrame에 [1,2,3]객체를 apply
print(df.apply(lambda x :[1,2,3]))

      col1  col2  col3
row1     1     1     1
row2     2     2     2
row3     3     3     3


In [11]:
#result_type='expand' func기준으로 배열 형태를 기준으로 확장하여 columns를 지정하게 된다
print(df.apply(lambda x: [1,2,3], axis=1, result_type='expand'))

      0  1  2
row1  1  2  3
row2  1  2  3
row3  1  2  3


In [12]:
#result_type ='reduce' func기준으로 축소하여 columns없이 그대로 series 객체로 반환함.
print(df.apply(lambda x :[1,2,3], axis=1, result_type='reduce'))

row1    [1, 2, 3]
row2    [1, 2, 3]
row3    [1, 2, 3]
dtype: object


In [13]:
#result_type = 'broadcast' func 기준으로 확장하되, columns는 기존 dataframe의 것을 사용함.
print(df.apply(lambda x : [1,2,3], axis=1, result_type='broadcast'))

      col1  col2  col3
row1     1     2     3
row2     1     2     3
row3     1     2     3


# 0302 요소별(applymap)
- 각 개체의 각 요소에 함수 적용하는 메서드

In [14]:
col = ['col1', 'col2', 'col3']
row = ['row1', 'row2', 'row3']
data =[[1,2,3],[4,5,6],[7,pd.NA,9]]
df =pd.DataFrame(data=data, index=row, columns=col)
print(df)

      col1  col2  col3
row1     1     2     3
row2     4     5     6
row3     7  <NA>     9


In [15]:
#함수 정용시 각 요소에 대해 함수의 연ㅇ산이 됨
print(df.applymap(lambda x : x**2, na_action='ignore'))

      col1  col2  col3
row1     1     4     9
row2    16    25    36
row3    49  <NA>    81


# 0303 함수 내 함수 연속적용(pipe)
- pipe 메서드는 함수를 연속적으로 사용할 때 유용한 메서드
- 함수가 인수를 사용할 때 pipe메서드를 사용하면 보다 직관적으로 적용할 수 있음
- func함수
- arg : 함수의 인수
- kwargs : dict 형태의 함수의 인수

In [16]:
org_data = pd.DataFrame({'info':['삼성전자/3/70000','SK하이닉스/2/100000']})
print(org_data)

              info
0     삼성전자/3/70000
1  SK하이닉스/2/100000


In [19]:
def code_name(data):
    result = pd.DataFrame(columns=['name', 'count', 'price'])
    df=pd.DataFrame(list(data['info'].str.split('/'))) #/ 로 구분하여 문자열 나누러 리스트에 넣음
    result['name']=df[0]
    result['count']=df[1]
    result['price']=df[2]
    result = result.astype({'count':int, 'price':int})
    return result

print(code_name(org_data))

     name  count   price
0    삼성전자      3   70000
1  SK하이닉스      2  100000


In [20]:
def value_cal(data, unit=''):
    result = pd.DataFrame(columns=['name', 'value'])
    result['name'] = data['name']
    result['value'] = data['count']*data['price']
    result = result.astype({'value':str})
    result['value']=result['value']+unit
    return result
input = code_name(org_data)
print(value_cal(input, '원'))

     name    value
0    삼성전자  210000원
1  SK하이닉스  200000원


In [21]:
#pipe 함수를 사용하지 않으면
print(value_cal(code_name(org_data), '원'))

     name    value
0    삼성전자  210000원
1  SK하이닉스  200000원


In [22]:
print(org_data.pipe(code_name).pipe(value_cal, '원'))

     name    value
0    삼성전자  210000원
1  SK하이닉스  200000원


# 0304 함수연속적용_축별
- 여러개의 함수를 동시에 적용할 수 있다.


In [2]:
import pandas as pd
import numpy as np

In [3]:
df= pd.DataFrame([[1,4,7], [2,5,8], [3,6,9]])
print(df)

   0  1  2
0  1  4  7
1  2  5  8
2  3  6  9


In [4]:
#np함수인 경우
ex1 = df.agg(np.prod)
print(ex1)

0      6
1    120
2    504
dtype: int64


In [5]:
#문자열인 경우
ex2 = df.agg('prod')
print(ex2)

0      6
1    120
2    504
dtype: int64


In [6]:
#lambda함수나 사용자 정의 함수를 사용할 수 있음
ex3 = df.agg([lambda x : min(x) * max(x)])
print(ex3)

          0   1   2
<lambda>  3  24  63


In [7]:
def func_sub(input):
    return max(input) - min(input)

ex4 = df.agg([func_sub, 'sum'])
print(ex4)

          0   1   2
func_sub  2   2   2
sum       6  15  24


In [8]:
# __name__메서드를 통해 따로 설정해주면 그 이름이 쓰임
def func_sub(input):
    return max(input)-min(input)
func_sub.__nam__ = '내함수'
ex5 = df.agg([func_sub, 'sum'])
print(ex5)

          0   1   2
func_sub  2   2   2
sum       6  15  24


In [10]:
#여러 개의 함수를 적용한다.
ex6 = df.agg(['min', 'max', 'sum', 'prod'])
print(ex6)

      0    1    2
min   1    4    7
max   3    6    9
sum   6   15   24
prod  6  120  504


In [11]:
#dict를 이용하는 경우 순서를 변경하는 것도 가능함
ex7 = df.agg({2:'sum', 0:'max', 1:'min'})
print(ex7)

2    24
0     3
1     4
dtype: int64


In [12]:
#dict함수 내에 다중함수를 적용할 수 있음. 이 경우 해당되지 않는 index는 NaN을 출력
ex8 = df.agg({0:['sum', 'prod'], 1: ['max', 'min'], 2:'mean'})
print(ex8)

        0    1    2
sum   6.0  NaN  NaN
prod  6.0  NaN  NaN
max   NaN  6.0  NaN
min   NaN  4.0  NaN
mean  NaN  NaN  8.0


In [13]:
#axis인수를 변경할 경우
ex2 = df.agg('prod', axis=0)
print(ex2)

0      6
1    120
2    504
dtype: int64


In [15]:
ex3 = df.agg('prod', axis=1)
print(ex3)

0     28
1     80
2    162
dtype: int64


# 0305 함수연속적용_요소별(transfor)
- agg와 비슷하게 함수를 적용하는 메서드

In [16]:
col = ['col1','col2','col3']
row = ['row1','row2','row3']
df = pd.DataFrame(data=[[10,40,70],[20,50,80],[30,60,90]],index=row,columns=col)
print(df)

      col1  col2  col3
row1    10    40    70
row2    20    50    80
row3    30    60    90


In [17]:
ex1 = df.transform(np.sqrt)
print(ex1)

          col1      col2      col3
row1  3.162278  6.324555  8.366600
row2  4.472136  7.071068  8.944272
row3  5.477226  7.745967  9.486833


In [18]:
ex2 = df.transform('sqrt')
print(ex2)

          col1      col2      col3
row1  3.162278  6.324555  8.366600
row2  4.472136  7.071068  8.944272
row3  5.477226  7.745967  9.486833


In [19]:
ex3 = df.transform(lambda x : np.sqrt(x))
print(ex3)

          col1      col2      col3
row1  3.162278  6.324555  8.366600
row2  4.472136  7.071068  8.944272
row3  5.477226  7.745967  9.486833


In [20]:
ex4 = df.transform(['exp', 'sqrt'])
print(ex4)

              col1                    col2                    col3          
               exp      sqrt           exp      sqrt           exp      sqrt
row1  2.202647e+04  3.162278  2.353853e+17  6.324555  2.515439e+30  8.366600
row2  4.851652e+08  4.472136  5.184706e+21  7.071068  5.540622e+34  8.944272
row3  1.068647e+13  5.477226  1.142007e+26  7.745967  1.220403e+39  9.486833


In [21]:
ex5 = df.transform({'col2':'exp', 'col1':'sqrt'})
print(ex5)

              col2      col1
row1  2.353853e+17  3.162278
row2  5.184706e+21  4.472136
row3  1.142007e+26  5.477226


# 0306 문자열 형식의 계산식 적용(eval메서드)
- expr : 문자열 형태의 게산임
- inplace : {True/False} 계산된 값이 원본을 변경할지의 여부

In [22]:
data = [[1,2,3],[4,5,6],[7,8,9]]
col = ['col1','col2','col3']
row = ['row1','row2','row3']
df = pd.DataFrame(data = data, index = row, columns= col)
print(df)

      col1  col2  col3
row1     1     2     3
row2     4     5     6
row3     7     8     9


In [23]:
print(df.eval('col4=col1*col2-col3'))

      col1  col2  col3  col4
row1     1     2     3    -1
row2     4     5     6    14
row3     7     8     9    47


In [24]:
print(df) #원본은 변경되지 않음. 

      col1  col2  col3
row1     1     2     3
row2     4     5     6
row3     7     8     9


In [25]:
#inplace=True인 경우, 원본이 변경됨.
print(df.eval('col4=col1*col2-col3', inplace=True))

None


In [26]:
print(df) #원본은 변경됨.

      col1  col2  col3  col4
row1     1     2     3    -1
row2     4     5     6    14
row3     7     8     9    47
